In [ ]:
import pandas as pd
import os
from tqdm import tqdm  # 진행 상황 확인을 위한 라이브러리

output_rows = []

# 1부터 4까지 그룹, 각 그룹에 대해 1부터 2500까지 파일 처리
for i in range(1, 5):
    for j in tqdm(range(1, 2501), desc=f"Processing group {i}"):
        file_path = f"../Data files/{i}_data files/result_{j}.csv"
        # 첫 번째 열은 인덱스 역할이므로 index_col=0으로 처리
        df = pd.read_csv(file_path, index_col=0)
        # static 값: "BW", "EGFR", "SEX", "RAAS", "BPS", "amt", "II"
        static_values = df.iloc[0][["BW", "EGFR", "SEX", "RAAS", "BPS", "amt", "II"]]
        # t=0인 row는 CP=0이므로 제외, time>=1인 경우의 CP값만 배열로 수집
        cp_sequence = df.loc[df["time"] >= 1, "CP"].tolist()
        # 한 행의 데이터 구성
        row_dict = static_values.to_dict()
        row_dict["CP_sequence"] = cp_sequence
        output_rows.append(row_dict)

# 최종 DataFrame 생성 (총 4×2500 = 10,000행)
result_df = pd.DataFrame(output_rows)
# Parquet 파일로 저장
result_df.to_parquet("../processed_data.parquet", engine="pyarrow")


Processing group 4: 100%|██████████| 2500/2500 [00:08<00:00, 287.86it/s]


In [1]:
import pandas as pd

df = pd.read_parquet("../processed_data.parquet", engine="pyarrow")
print(df.head())

       BW    EGFR  SEX  RAAS  BPS    amt     II  \
0   68.28   66.37  1.0   1.0  1.0  244.0  482.0   
1   74.65   73.51  1.0   0.0  1.0  349.0  333.0   
2  109.14  112.20  0.0   0.0  0.0  253.0  314.0   
3   80.45   80.02  0.0   0.0  0.0  414.0  383.0   
4   81.58   81.29  1.0   0.0  1.0  210.0  387.0   

                                         CP_sequence  
0  [47.8230483978772, 47.4611821136084, 47.106996...  
1  [61.7256850782601, 61.2639927876879, 60.810790...  
2  [90.7251063784226, 89.513886832227, 88.3331084...  
3  [148.141296157798, 145.663902299757, 143.24205...  
4  [102.131438124903, 100.003249264386, 97.935064...  
